In [2]:
from github import Github
import pandas as pd

In [3]:
%run settings.ipynb

token = %env token

github = Github(token)

env: token=a96bcae252e327cdc0f1ea9a4f91e1fb2b5a7a4c


## Análise de testes usados nos projetos do dataset.

1. Análise de quantos projetos usam arquivos de controle.
2. Armazenar arquivos de controle.
3. Verificar se existe algum pacote de teste no arquivo de controle.
4. Verificar qual pacote está sendo usado.


In [11]:
tools = pd.read_csv('dataset/de_para_tools.csv', sep=';')

tools['language'].unique()


array(['JavaScript', 'Python', 'Java', 'Go', 'Ruby', 'PHP', 'TypeScript',
       'C#'], dtype=object)

In [44]:
manage_file = {}

for t in tools['language'].unique():
    manage_file[t] = {"yes": 0, "total": 0}

manage_file

{'C#': {'total': 0, 'yes': 0},
 'Go': {'total': 0, 'yes': 0},
 'Java': {'total': 0, 'yes': 0},
 'JavaScript': {'total': 0, 'yes': 0},
 'PHP': {'total': 0, 'yes': 0},
 'Python': {'total': 0, 'yes': 0},
 'Ruby': {'total': 0, 'yes': 0},
 'TypeScript': {'total': 0, 'yes': 0}}

### TO-DO: Atualização Dataset.

Filtro para restringir dataset para apenas a lista de linguagens definidas.

In [40]:
repo_full = pd.read_csv('dataset/repositories.csv', sep=';')
#repo_full.drop("Unnamed: 0", axis=1, inplace=True) 
repo_full.head()

,full_name,git_url,id,language
0,b4winckler/macvim,git://github.com/b4winckler/macvim.git,692798,C
1,php/php-src,git://github.com/php/php-src.git,1903522,C
2,antirez/redis,git://github.com/antirez/redis.git,156018,C
3,libgit2/libgit2,git://github.com/libgit2/libgit2.git,901662,C
4,torvalds/linux,git://github.com/torvalds/linux.git,2325298,C


### Quantidade total de projetos de uma mesma linguagem.

In [45]:
for index, row in repo_full.iterrows():
    if row['language'] in manage_file:
        manage_file[row['language']]["total"]+=1

manage_file

{'C#': {'total': 13, 'yes': 0},
 'Go': {'total': 33, 'yes': 0},
 'Java': {'total': 38, 'yes': 0},
 'JavaScript': {'total': 81, 'yes': 0},
 'PHP': {'total': 21, 'yes': 0},
 'Python': {'total': 52, 'yes': 0},
 'Ruby': {'total': 23, 'yes': 0},
 'TypeScript': {'total': 15, 'yes': 0}}

### Quantidade total de projetos que possui o arquivo de gerencia de pacotes.

In [46]:
for index, row in repo_full.iterrows():
    packages = tools[tools['language'] == row['language']]
    arq = packages['file_packages_installed'].unique()

    if(len(arq) > 0):
        contents = github.get_repo(row['full_name']).get_contents("")

        for content in contents:
            try:
                if(str(content.path) == str(arq[0])):
                    #Registrar quantidade de projetos possui arquivo de controle.
                    manage_file[row['language']]["yes"]+=1
                    
                    #Obter conteúdo do arquivo.

            except UnicodeEncodeError:
                pass

manage_file           
        

{'C#': {'total': 13, 'yes': 7},
 'Go': {'total': 33, 'yes': 16},
 'Java': {'total': 38, 'yes': 21},
 'JavaScript': {'total': 81, 'yes': 73},
 'PHP': {'total': 21, 'yes': 19},
 'Python': {'total': 52, 'yes': 16},
 'Ruby': {'total': 23, 'yes': 1},
 'TypeScript': {'total': 15, 'yes': 15}}